<a href="https://colab.research.google.com/github/reic/colab_python/blob/main/crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 網路爬蟲與多執行緒練習



In [ ]:
#@title UU看書網 
#@markdown 還在修正的程式，可以直接從這一個區塊執行


import requests
import os,re
from bs4 import BeautifulSoup
import concurrent.futures

try:
  os.mkdir("/content/tmp")
except:
  print("目錄已存在")
os.chdir("/content/tmp")
os.system("rm -fr *")


def get_html(urls):
  [title,art_url]=urls
  art_id=art_url[art_url.rfind("/")+1:-5]
  
  soup=BeautifulSoup(requests.get(art_url).text)
  content=soup.find(name="div",id='contentbox')
  print(art_id)
  text=f"# {title}\n\n"
  context=str(content).replace("\n","").replace("\r","")
  context=context.replace("\xa0\xa0\xa0\xa0","")
  context=re.sub('<div class="ad_content".*?</div>','',context)
  context=context.replace("<br/>","\n").replace("</p>","\n")
  context=re.sub('<.*?>',"",context).split("\n")
  context=[itm.strip() for itm in context if len(itm)>0]
  text+="\n\n".join(context)+"\n\n"

  
  
  with open(f"{art_id}.txt",mode="w",encoding="utf-8") as f:
    f.write(text)
  

#@markdown 書籍目錄網址
url="https://tw.uukanshu.com/b/47158/" #@param {type:'string'}
title="\u5929\u9053\u5716\u66F8\u9928" #@param {type:"string"}
author="\u6A6B\u6383\u5929\u6DAF" #@param {type:"string"}
#@markdown 打勾，將會直接變成 epub
file2epub = True #@param {type:"boolean"}

# 標題設定義
YAML=f'''---
title: {title}
author: {author}
language: zh-Hant
---'''

with open("title.txt",mode="w",encoding='utf-8') as f:
  f.write(YAML)

sites=url[:url.find("/",8)]
# sites=url[:url.rfind("/")]

reg=requests.get(url)
# soup=BeautifulSoup(reg.text,"html.parser")
soup=BeautifulSoup(reg.text)
output_name=soup.find("h2").getText()
articles=soup.find(name="ul",id="chapterList").find_all("a")

links=[]
# len(articles)
for i in articles:
  href=i.get("href")
  links.append([i.get("title"),f"{sites}{href}"])
# links.sort(key=lambda x: x[1])
files_text=[link[1][link[1].rfind("/")+1:link[1].rfind(".")]+".txt" for link in links]

# 同時建立及啟用10個執行緒
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    executor.map(get_html, links)

output_name=soup.find("h2").getText()
# files_text=os.listdir()
# files_text=[file for file in files_text if file.endswith(".txt")]
# 檔案排序，需要考慮 檔案名稱長短不一的問題，問前是透過數字的處理
# files_text.sort(key=lambda x:int(x[:-4]))
if file2epub:
  mdfiles=[ itm for itm in files_text[::-1] ]
  os.system("pandoc -o \"../{}.epub\" title.txt {}".format(title," ".join(mdfiles)))
  from google.colab import files
  files.download('../{}.epub'.format(title)) 
  pass
else:
  with open(f"../{output_name}.txt","w",encoding='utf-8') as f:
    for file in files_text[::-1]:
      with open(file,"r") as f2:
        f.write(f2.read())
  from google.colab import files
  files.download('../{}.txt'.format(output_name)) 



In [ ]:
#@title 飄天文學 
#@markdown 還在修正的程式，可以直接從這一個區塊執行
def check_package(itm):
  import importlib
  try:
    importlib.import_module(itm)
    print(f"{itm} 套件已經安裝")
  except ImportError:
    print(f"{itm} 套件尚未安裝，正在安裝中...")
    !pip install {itm}


check_package("inlp")


import requests
import os,re
from bs4 import BeautifulSoup
import concurrent.futures
from inlp.convert import chinese

try:
  os.mkdir("/content/tmp")
except:
  print("目錄已存在")
os.chdir("/content/tmp")
os.system("rm -fr *")


def get_html(urls):
  [title,art_url]=urls
  art_id=art_url[art_url.rfind("/")+1:-5]
  # print(art_url)
  req=requests.get(art_url)
  req.encoding="gbk"
  soup=BeautifulSoup(req.text)
  # content=soup.find('div', {'id': 'content', 'class': 'fonts_mesne'})
  
  print(art_id)


  # Find all the siblings of the table element up to the center element
  content = []
  content=soup.body
  for tag in content.find_all(['div', 'h1','table','script','center']):
    tag.extract()

  text=f"# {title}\n\n"
  context=str(content).replace("\n","").replace("\r","")
  context=context.replace("\xa0\xa0\xa0\xa0","")
  context=re.sub('<div class="ad_content".*?</div>','',context)
  context=context.replace("<br/>","\n").replace("</p>","\n")
  context=re.sub('<.*?>',"",context).split("\n")
  context=[itm.strip() for itm in context if len(itm)>0]
  text+="\n\n".join(context)+"\n\n"
  text=chinese.s2t(text)

  
  
  with open(f"{art_id}.txt",mode="w",encoding="utf-8") as f:
    f.write(text)
  

#@markdown 書籍目錄網址
url="https://www.ptwxz.com/html/12/12539/" #@param {type:'string'}
title="\u6211\u7684\u5E2B\u5085\u6BCF\u5230\u5927\u9650\u624D\u7A81\u7834" #@param {type:"string"}
author="\u8C6C\u8089200\u65A4 " #@param {type:"string"}
#@markdown 打勾，將會直接變成 epub
file2epub = True #@param {type:"boolean"}

# 標題設定義
YAML=f'''---
title: {title}
author: {author}
language: zh-Hant
---'''

with open("title.txt",mode="w",encoding='utf-8') as f:
  f.write(YAML)

sites=url
# sites=url[:url.rfind("/")]

reg=requests.get(url)
reg.encoding = 'gbk'
# soup=BeautifulSoup(reg.text,"html.parser")
soup=BeautifulSoup(reg.text)

articles=soup.find(name="div",class_="centent").find_all("a")

links=[]
# len(articles)
for i in articles:
  href=i.get("href")
  links.append([chinese.s2t(i.text),f"{sites}/{href}"])
files_text=[link[1][link[1].rfind("/")+1:link[1].rfind(".")]+".txt" for link in links]

# 同時建立及啟用10個執行緒
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    executor.map(get_html, links)

output_name=title
# files_text=os.listdir()
# files_text=[file for file in files_text if file.endswith(".txt")]
# 檔案排序，需要考慮 檔案名稱長短不一的問題，問前是透過數字的處理
# files_text.sort(key=lambda x:int(x[:-4]))
if file2epub:
  mdfiles=[ itm for itm in files_text]
  os.system("pandoc -o \"../{}.epub\" title.txt {}".format(title," ".join(mdfiles)))
  from google.colab import files
  files.download('../{}.epub'.format(title)) 
  pass
else:
  with open(f"../{output_name}.txt","w",encoding='utf-8') as f:
    for file in files_text:
      with open(file,"r") as f2:
        f.write(f2.read())
  from google.colab import files
  files.download('../{}.txt'.format(output_name)) 



In [ ]:
#@title 69書吧
#@markdown 還在修正的程式，可以直接從這一個區塊執行
def check_package(itm):
  import importlib
  try:
    importlib.import_module(itm)
    print(f"{itm} 套件已經安裝")
  except ImportError:
    print(f"{itm} 套件尚未安裝，正在安裝中...")
    !pip install {itm}


check_package("inlp")


import requests
import os,re
from bs4 import BeautifulSoup
import concurrent.futures
from inlp.convert import chinese

try:
  os.mkdir("/content/tmp")
except:
  print("目錄已存在")
os.chdir("/content/tmp")
os.system("rm -fr *")


def get_html(urls):
  [title,art_url]=urls
  art_id=art_url[art_url.rfind("/")+1:]
  # print(art_url)
  req=requests.get(art_url)
  req.encoding="gbk"
  soup=BeautifulSoup(req.text)
  
  
  print(art_id)


  # Find all the siblings of the table element up to the center element
  content = []
  content=soup.find(name="div",class_="txtnav")
  
  for tag in content.find_all(['div','h1','script','center']):
    tag.extract()
  

  text=f"# {title}\n\n"
  context=str(content).replace("\n","").replace("\r","")
  context=context.replace("\xa0\xa0\xa0\xa0","")
  context=re.sub('<div class="ad_content".*?</div>','',context)
  context=context.replace("<br/>","\n").replace("</p>","\n")
  context=re.sub('<.*?>',"",context).split("\n")
  context=[itm.strip() for itm in context if len(itm)>0]
  text+="\n\n".join(context)+"\n\n"
  text=chinese.s2t(text)

  # print(text)
    
  with open(f"{art_id}.txt",mode="w",encoding="utf-8") as f:
    f.write(text)
  

#@markdown 書籍目錄網址
url="https://www.69shu.com/25102/" #@param {type:'string'}
title="\u8A85\u4ED9" #@param {type:"string"}
author="\u856D\u9F0E" #@param {type:"string"}
#@markdown 打勾，將會直接變成 epub
file2epub = True #@param {type:"boolean"}

# 標題設定義
YAML=f'''---
title: {title}
author: {author}
language: zh-Hant
---'''

with open("title.txt",mode="w",encoding='utf-8') as f:
  f.write(YAML)

sites=url
# sites=url[:url.rfind("/")]

reg=requests.get(url)
reg.encoding = 'gbk'
# soup=BeautifulSoup(reg.text,"html.parser")
soup=BeautifulSoup(reg.text)

articles=soup.find(name="div",id="catalog").find_all("a")
articles[:4]
links=[]
# # len(articles)
for i in articles:
  href=i.get("href")
  links.append([chinese.s2t(i.text),href])

files_text=[link[1][link[1].rfind("/")+1:]+".txt" for link in links]
# print(links[0])
# get_html(links[0])
# 同時建立及啟用10個執行緒
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    executor.map(get_html, links)

output_name=title
# files_text=os.listdir()
# files_text=[file for file in files_text if file.endswith(".txt")]
# 檔案排序，需要考慮 檔案名稱長短不一的問題，問前是透過數字的處理
# files_text.sort(key=lambda x:int(x[:-4]))
if file2epub:
  mdfiles=[ itm for itm in files_text]
  os.system("pandoc -o \"../{}.epub\" title.txt {}".format(title," ".join(mdfiles)))
  from google.colab import files
  files.download('../{}.epub'.format(title)) 
  pass
else:
  with open(f"../{output_name}.txt","w",encoding='utf-8') as f:
    for file in files_text:
      with open(file,"r") as f2:
        f.write(f2.read())
  from google.colab import files
  files.download('../{}.txt'.format(output_name)) 

In [ ]:
import importlib
def check_package(itm): 
  try:
    importlib.import_module(itm)
    print(f"{itm} 套件已經安裝")
  except ImportError:
    print(f"{itm} 套件尚未安裝，正在安裝中...")
    !pip install {itm}


check_package("inlp")

from inlp.convert import chinese

# mdfiles=[ itm for itm in files_text]
# os.system("pandoc -o \"../{}.epub\" title.txt {}".format(title," ".join(mdfiles)))
# from google.colab import files
# files.download('../{}.epub'.format(title)) 

In [ ]:
#@title 微風看書
#@markdown 還在修正的程式，可以直接從這一個區塊執行


import requests
import os,re
from bs4 import BeautifulSoup
import concurrent.futures

try:
  os.mkdir("/content/tmp")
except:
  print("目錄已存在")
os.chdir("/content/tmp")
os.system("rm -fr *")


def get_html(urls):
  [title,art_url]=urls
  art_id=art_url[art_url.rfind("/")+1:]
  
  
  soup=BeautifulSoup(requests.get(art_url).text)
  pages=soup.find("h1").getText()
  chk=pages.rfind("/")
  if chk>0:
    pagenum=pages[pages.rfind("/")+1:-1]
    content=list(soup.find("div",id="content").find_all("p"))
    if int(pagenum) > 1:
      for i in range(2,int(pagenum)+1):
        # print(get_html_page(urls,i)[:3])
        content.extend(get_html_page(urls,i))
    # 將 bs object 轉換為 string
    content=" ".join([str(itm) for itm in content])
  else:
    content=soup.find("div",id="content").getText()
  print(art_id,pagenum)
  
  
  text=f"# {title}\n\n"
  context=str(content).replace("\n","").replace("\r","")
  context=context.replace("\xa0\xa0\xa0\xa0","")
  context=re.sub('<div class="ad_content".*?</div>','',context)
  context=context.replace("<br/>","\n").replace("</p>","\n")
  context=re.sub('<.*?>',"",context).split("\n")
  context=[itm.strip() for itm in context if len(itm)>0]
  text+="\n\n".join(context)+"\n\n"
  


  with open(f"{art_id}.txt",mode="w",encoding="utf-8") as f:
    f.write(text)


def get_html_page(urls,num):
  [title,art_url]=urls
  pageurl=f"{art_url}/{num}.html"
  
  soup=BeautifulSoup(requests.get(pageurl).text)
  content=list(soup.find("div",id="content").find_all("p"))
  
  return content
  

#@markdown 書籍目錄網址
url="https://www.wfxs.tw/booklist/891893.html" #@param {type:'string'}
title="\u5927\u590F\u6587\u8056" #@param {type:"string"}
author="\u4E03\u6708\u672A\u6642" #@param {type:"string"}
#@markdown 打勾，將會直接變成 epub
file2epub = True #@param {type:"boolean"}

# 標題設定義
YAML=f'''---
title: {title}
author: {author}
language: zh-Hant
---'''

with open("title.txt",mode="w",encoding='utf-8') as f:
  f.write(YAML)

sites=url[:url.find("/",8)]
# sites=url[:url.rfind("/")]

reg=requests.get(url)
soup=BeautifulSoup(reg.text)
articles=soup.find(name="div",id="readerlists").find_all("a")


links=[]
len(articles)
for i in articles:
  href=i.get("href")[:-1]
  itmTitle=i.get("title")[i.get("title").find(".")+1:]
  links.append([itmTitle,f"{sites}{href}"])
# links.sort(key=lambda x: x[1])


files_text=[link[1][link[1].rfind("/")+1:]+".txt" for link in links]


# 同時建立及啟用10個執行緒
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    executor.map(get_html, links)

output_name=title
listfiles=os.listdir()
mdfiles=[ itm for itm in files_text if itm in listfiles]
# 檔案排序，需要考慮 檔案名稱長短不一的問題，問前是透過數字的處理
# files_text.sort(key=lambda x:int(x[:-4]))
if file2epub:
  # mdfiles=[ itm for itm in files_text if itm in listfiles]
  os.system("pandoc -o \"../{}.epub\" title.txt {}".format(title," ".join(mdfiles)))
  from google.colab import files
  files.download('../{}.epub'.format(title)) 
  pass
else:
  with open(f"../{output_name}.txt","w",encoding='utf-8') as f:
    for file in mdfiles:
      with open(file,"r") as f2:
        f.write(f2.read())
  from google.colab import files
  files.download('../{}.txt'.format(output_name)) 



In [ ]:
#@title 稷下書院
#@markdown 


import requests
import os,re
from bs4 import BeautifulSoup
import concurrent.futures

try:
  os.mkdir("/content/tmp")
except:
  print("目錄已存在")
os.chdir("/content/tmp")
os.system("rm -fr *")


def get_html(urls):
  [title,art_url]=urls
  art_id=art_url[art_url.rfind("/")+1:]
  
  
  soup=BeautifulSoup(requests.get(art_url).text)
  print(art_id)
  content=soup.find("div",class_="content")
  advernum=content.find_all("div")
  for itm in advernum:
    itm.clear()   

  
  text=f"# {title}\n\n"
  context=str(content).replace("\n","").replace("\r","")
  context=context.replace("\xa0\xa0\xa0\xa0","")
  context=re.sub('<div class="ad_content".*?</div>','',context)
  context=context.replace("<br/>","\n").replace("</p>","\n")
  context=re.sub('<.*?>',"",context).split("\n")
  context=[itm.strip() for itm in context if len(itm)>0]
  text+="\n\n".join(context)+"\n\n"

  # print(text)


  with open(f"{art_id}.txt",mode="w",encoding="utf-8") as f:
    f.write(text)



  

#@markdown 書籍目錄網址
url="https://www.novel543.com/1004298677/dir" #@param {type:'string'}
title="\u5927\u590F\u6587\u8056" #@param {type:"string"}
author="\u4E03\u6708\u672A\u6642" #@param {type:"string"}
#@markdown 打勾，將會直接變成 epub
file2epub = True #@param {type:"boolean"}

# 標題設定義
YAML=f'''---
title: {title}
author: {author}
language: zh-Hant
---'''

with open("title.txt",mode="w",encoding='utf-8') as f:
  f.write(YAML)

sites=url[:url.find("/",8)]
# sites=url[:url.rfind("/")]

reg=requests.get(url)
soup=BeautifulSoup(reg.text)
articles=soup.find(name="div",class_="read").find("dl").find_next("dl").find_all("a")


links=[]
for i in articles:
  href=i.get("href")
  itmTitle=i.text
  links.append([itmTitle,f"{sites}{href}"])
# links.sort(key=lambda x: x[1])

files_text=[link[1][link[1].rfind("/")+1:]+".txt" for link in links]


# 同時建立及啟用10個執行緒
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    executor.map(get_html, links)

output_name=title

if file2epub:
  mdfiles=[ itm for itm in files_text ]
  os.system("pandoc -o \"../{}.epub\" title.txt {}".format(title," ".join(mdfiles)))
  from google.colab import files
  files.download('../{}.epub'.format(title)) 
  pass
else:
  with open(f"../{output_name}.txt","w",encoding='utf-8') as f:
    for file in files_text[::-1]:
      with open(file,"r") as f2:
        f.write(f2.read())
  from google.colab import files
  files.download('../{}.txt'.format(output_name)) 



# 測試後，暫時沒有使用的程式碼區塊

In [ ]:
#@title 測試UU看書 專用(多執行緖) 
#@markdown 



import requests
import os,re
from bs4 import BeautifulSoup
import concurrent.futures

try:
  os.mkdir("/content/tmp")
except:
  print("目錄已存在")
os.chdir("/content/tmp")
os.system("rm -fr *")


def get_html(urls):
  [title,art_url,art_id]=urls
  # art_id=art_url[art_url.rfind("/")+1:-5]
  
  
  soup=BeautifulSoup(requests.get(art_url).text)
  title=soup.find("h3").getText()
  

  context=soup.find(name="div",id='bookContent').find_all("p")
  context=[itm.getText().strip() for itm in context if len(itm.getText().strip()) >0 ]
  print(art_id)

  text=f"{title}\n\n"
  text+="\n\n".join(context)+"\n\n"
  

  # context=str(content).replace("\n","").replace("\r","")
  # context=context.replace("\xa0\xa0\xa0\xa0","")
  # context=re.sub('<div class="ad_content".*?</div>','',context)
  # context=context.replace("<br/>","\n").replace("</p>","\n")
  # context=re.sub('<.*?>',"",context).split("\n")
  # context=[itm.strip() for itm in context if len(itm)>0]
  # text+="\n\n".join(context)+"\n\n"
  
  with open(f"{art_id}.txt",mode="w",encoding="utf-8") as f:
    f.write(text)
  

#@markdown 書籍目錄網址
url="https://tw.uukanshu.com/b/154415/" #@param {type:'string'}
sites=url[:url.rfind("/")]


reg=requests.get(url)
# soup=BeautifulSoup(reg.text,"html.parser")
soup=BeautifulSoup(reg.text)
output_name=soup.find("h1").getText()
output_name +=" "+ soup.find(name="div",class_="book-info").find("dd").getText()
#output_name
articles=soup.find(name="ul",id="chapterList").find_all("a")
links=[]
# len(articles)
for i in articles:
  href=i.get("href")
  # links.append([i.get("title"),f"{sites}{href}",i.get("name")])
  links.append([i.getText(),f"{sites}/{href}",int(i.get("name"))])

links.sort(key=lambda x: x[2])

# print(links[0])
# get_html(links[0])
# 同時建立及啟用10個執行緒
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    executor.map(get_html, links)


files_text=os.listdir()
files_text=[file for file in files_text if file.endswith(".txt")]
# 檔案排序，需要考慮 檔案名稱長短不一的問題，問前是透過數字的處理
files_text.sort(key=lambda x:int(x[:-4]))
with open(f"../{output_name}.txt","w",encoding='utf-8') as f:
  for file in files_text:
    with open(file,"r") as f2:
      f.write(f2.read())

from google.colab import files
files.download('../{}.txt'.format(output_name)) 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 效率

透過下述的方法，合併檔案，因為輸出檔需要被反覆的開始太多次，隨著檔案大小逐漸增加。讓效能下跌

```python
for file in files:
  os.system("cat {}>> ../{}.txt".format(file,output_name))
```
若改用下述的方法， output 檔，只需要開啟一次。可以大大縮短時間。

```python
with open(f"../{output_name}.txt","w",encoding='utf-8') as f:
  for file in files_text:
    with open(file,"r") as f2:
      f.write(f2.read())
```

In [ ]:
#@title 品書閣 專用(多執行緖)
#@markdown 還在修正的程式，可以直接從這一個區塊執行

import requests
import os,re
from bs4 import BeautifulSoup
import concurrent.futures

try:
  os.mkdir("/content/tmp")
except:
  print("目錄已存在")
os.chdir("/content/tmp")
os.system("rm -fr *")

stop_word="請大家收藏" 

  
def get_html(arr):
  
  [titles,links]=arr 
  art_id=links[links.rfind("/")+1:-5]

  soup=BeautifulSoup(requests.get(links).text)
  content=soup.find(name="div",id='content').find_all("p")

  text=f"{titles}\n"

  for itm in content[1:]:
    txt=itm.getText()
    if "一秒記住" in txt:
      continue
    if "點下一章繼續閱讀" in txt:
      text+=f"{txt}\n"
      break
    if stop_word in txt:
      break    
    # text+="%s \n" %txt
    
    text+=f"{txt}\n"
  
  # print(art_id)
  next_article=BeautifulSoup(requests.get(links).text).find("a",text="下一章").get("href")
  

  if art_id in next_article:
    
    soup2=BeautifulSoup(requests.get(f"{sites}{next_article}").text)
    content2=soup2.find(name="div",id="content").find_all("p")
    print("***",next_article)
    for itm2 in content2:
      txt2=itm2.getText()
      if "一秒記住" in txt2:
        continue
      if stop_word in txt2:
        break          
      text+=f"{txt2}\n"
   
  with open(f"{art_id}.txt",mode="w",encoding="utf-8") as f:
    f.write(text)


#@markdown 書籍目錄網址
url="https://tw.pinsuge.com/index/85767.html" #@param {type:'string'}
sites=url[:url.find("/",8)]
req = requests.get(url)
soup=BeautifulSoup(req.text)
articles=soup.find_all("a")
links=[]
titles=[]
for itm in articles[1:]:
  if not itm.get("title"):
    continue 
  links.append([itm.get("title",),"{}{}".format(sites,itm.get("href"))])

# 同時建立及啟用10個執行緒
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    executor.map(get_html, links)

files_text=os.listdir()
files_text=[file for file in files_text if file.endswith(".txt")]
# 檔案排序，需要考慮 檔案名稱長短不一的問題，問前是透過數字的處理
files_text.sort(key=lambda x:int(x[:-4]))
with open(f"../{output_name}.txt","w",encoding='utf-8') as f:
  for file in files_text:
    with open(file,"r") as f2:
      f.write(f2.read())
      
from google.colab import files
files.download('../{}.txt'.format(output_name)) 


# 參考資料

In [ ]:
#@title BeauttifulSoup 練習
#@markdown extract, decompose 的練習
html_doc = """<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>


<p class="story">Once upon a time there were three little sisters; and their names were <script>.lkjlkjlj\n\r
我是\n</script> <i> hello world</i>
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story">...</p>
"""
from bs4 import BeautifulSoup
soup=BeautifulSoup(html_doc,"lxml")
txt=soup.find("p","story")
print(str(txt))
print("".center(100,"-"))
txt.script.decompose()

txt.a.extract()
txt.a.extract()

print(str(txt))

In [ ]:
#@title 品書閣 (舊) 單執行緖版本
#@markdown 查看目錄=>章節列表
import requests
import os,re,time
from bs4 import BeautifulSoup
import concurrent.futures

try:
  os.mkdir("/content/tmp")
except:
  print("目錄已存在")
os.chdir("/content/tmp")
os.system("rm -fr *")

url="https://tw.pinsuge.com/index/85767.html" #@param {type:'string'}
sites=url[:url.find("/",8)]
req = requests.get(url)
#@markdown 在 stop_word 之後為相關的廣告內容
stop_word="請大家收藏" #@param {type:'string'}
#@markdown 若章節分成兩個頁面的處理字元
next_word="點下一章繼續" #@param {type:'string'}
soup=BeautifulSoup(req.text)

soup
articles=soup.find_all(name="a")
output_name=soup.find("h1").getText()
links=[]

for itm in articles[1:]:
  if not itm.get("title"):
    continue 
  links.append([itm.get("title",),"{}{}".format(sites,itm.get("href"))])


def get_content(content):
  to_text=''
  for itm in content[:-1]:
    txt=itm.getText()
    if "一秒記住" in txt:
      continue
    if stop_word in txt:
      break    
    to_text+="%s \n" %txt
  return to_text

def get_soup(url):
  return BeautifulSoup(requests.get(url).text)

def cont_chapter_check(soup,text):
  return (len(soup.find_all(name="p",string=re.compile(text))) >0)

def cont_chapter_link(soup):
  return soup.find("a",text="下一章").get("href")

index=1
star=time.time()
for link in links:  
  soup=get_soup(link[1])
  to_text=''
  to_text+="%s \n" %link[0]
  print(link[0])
  to_text+=get_content(soup.find(name="div",id="content").find_all(name="p"))
  
  if cont_chapter_check(soup,next_word):
    reg2=requests.get("{}{}".format(sites,cont_chapter_link(soup)))
    to_text+=get_content(BeautifulSoup(reg2.text).find(name="div",id="content").find_all(name="p"))
  # print(to_text)

  with open("%03d.txt" % index,mode="w",encoding="utf-8") as f:
    f.write(to_text)
  index+=1

files=os.listdir()
files=[file for file in files if file.endswith(".txt")]
files.sort()
files
for file in files:
  os.system("cat {}>> ../{}.txt".format(file,output_name))

from google.colab import files
files.download('../{}.txt'.format(output_name)) 
end=time.time()
print("經過了 {end-time} 秒")

In [ ]:
#@title 多執行序參考程式範例

from bs4 import BeautifulSoup
import concurrent.futures
import requests
import time
 
 
def scrape(urls):
  
    response = requests.get(urls)
 
    soup = BeautifulSoup(response.content, "lxml")
 
    # 爬取文章標題
    titles = soup.find_all("h3", {"class": "post_title"})
 
    for title in titles:
        print(title.getText().strip())
 
    time.sleep(2)
 
 
base_url = "https://www.inside.com.tw/tag/AI"
urls = [f"{base_url}?page={page}" for page in range(1, 6)]  # 1~5頁的網址清單
print(urls) 
start_time = time.time()  # 開始時間
# scrape(urls)

 
# 同時建立及啟用10個執行緒
# with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
#     executor.map(scrape, urls)
 
end_time = time.time()
print(f"{end_time - start_time} 秒爬取 {len(urls)} 頁的文章")